In [2]:
import pandas as pd
import numpy as np

import Bio.PDB
from Bio import PDB
from Bio.PDB.PDBIO import PDBIO as io
import os

Ha valamiért nem importálná az egész PDB sublibrary-t
import Bio.SVDSuperimposer
from Bio.PDB import Superimposer
from Bio.PDB.PDBIO import PDBIO

In [3]:
basedir = "/home/takacsg/Documents/sh2db/data/"

# Import "Pfam" table:

### Without negative values

In [5]:
# pfam = pd.read_csv(basedir+'data/SH2_domain_containing_prot_right_resnum_with_pdb_nums_domcol_0503.csv')
pfam = pd.read_csv(basedir+'pfam_table_1017.csv')
pfam.drop(['Unnamed: 0'], 'columns', inplace = True)
#pfam.drop(['Unnamed: 0.1'], 'columns', inplace = True)
pfam

UniProt entry Gene_col Gene_domain Pole UniProt residues PDB ID  \
0      3BP2_HUMAN     3BP2      SH3BP2    N        458 - 538   2CR4   
1      ABL1_HUMAN     ABL1        ABL1    N        127 - 202   1AB2   
2      ABL1_HUMAN     ABL1        ABL1    N        127 - 202   1OPL   
3      ABL1_HUMAN     ABL1        ABL1    N        127 - 202   1OPL   
4      ABL1_HUMAN     ABL1        ABL1    N        127 - 202   2ABL   
..            ...      ...         ...  ...              ...    ...   
635   ZAP70_HUMAN    ZAP70     ZAP70_C    C        163 - 239   1M61   
636   ZAP70_HUMAN    ZAP70     ZAP70_C    C        164 - 239   2OQ1   
637   ZAP70_HUMAN    ZAP70     ZAP70_C    C        165 - 239   2OZO   
638   ZAP70_HUMAN    ZAP70     ZAP70_C    C        167 - 239   4XZ0   
639   ZAP70_HUMAN    ZAP70     ZAP70_C    C        168 - 239   4XZ1   

    PDB chain ID PDB residues Category Gene name  ... Res_stop  Uniprot_ID  \
0              A     20 - 100   SigReg    SH3BP2  ...      100      P78314   
1              A      11 - 86  Kinases      ABL1  ...       86      P00519   
2              A    146 - 221  Kinases      ABL1  ...      221      P00519   
3              B    146 - 221  Kinases      ABL1  ...      221      P00519   
4              A    146 - 221  Kinases      ABL1  ...      221      P00519   
..           ...          ...      ...       ...  ...      ...         ...   
635            A    163 - 239  Kinases     ZAP70  ...      239      P43403   
636            A    165 - 241  Kinases     ZAP70  ...      241      P43403   
637            A    163 - 239  Kinases     ZAP70  ...      239      P43403   
638            A    163 - 239  Kinases     ZAP70  ...      239      P43403   
639            A    163 - 239  Kinases     ZAP70  ...      239      P43403   

     New_uniprot_start New_uniprot_stop  Old_uniprot_start  Old_uniprot_stop  \
0                  435              561                458               538   
1                  105              228                127               202   
2                  105              228                127               202   
3                  105              228                127               202   
4                  105              228                127               202   
..                 ...              ...                ...               ...   
635                148              265                163               239   
636                148              265                164               239   
637                148              265                165               239   
638                148              265                167               239   
639                148              265                168               239   

     diff_start  New_PDB_start  diff_stop  New_PDB_stop  
0            23              0         23           123  
1            22              0         26           112  
2            22            124         26           247  
3            22            124         26           247  
4            22            124         26           247  
..          ...            ...        ...           ...  
635          15            148         26           265  
636          16            149         26           267  
637          17            146         26           265  
638          19            144         26           265  
639          20            143         26           265  

[640 rows x 22 columns]

# Import table from alignment:

In [6]:
table = pd.read_csv(basedir+'table_from_alignment_with_pdbs_0917.csv', index_col = [0,1])
table = table.fillna('')
table

2  3  4  5  6  7    8    9   10   11  ... 254 255 256  \
0       1                                                   ...               
NaN     NaN                                                 ...               
        NaN                                                 ...               
ABL1    nums                            105  106  107  108  ...   -   -   -   
        seq           -  -  -  -  -  -    K    N    G    Q  ...   -   -   -   
        1AB2_A_SH2_N                                        ...               
...                  .. .. .. .. .. ..  ...  ...  ...  ...  ...  ..  ..  ..   
ZAP70_N 2OQ1_A_SH2_N                                        ...               
        2OZO_A_SH2_N                                        ...               
        4K2R_A_SH2_N                                        ...               
        4XZ0_A_SH2_N                                        ...               
        4XZ1_A_SH2_N                                        ...               

                     257 258 259 260 261 262 263  
0       1                                         
NaN     NaN                                       
        NaN                                       
ABL1    nums           -   -   -   -   -   -      
        seq            -   -   -   -   -   -      
        1AB2_A_SH2_N                              
...                   ..  ..  ..  ..  ..  ..  ..  
ZAP70_N 2OQ1_A_SH2_N                              
        2OZO_A_SH2_N                              
        4K2R_A_SH2_N                              
        4XZ0_A_SH2_N                              
        4XZ1_A_SH2_N                              

[877 rows x 262 columns]

# Structural alignment:

In [7]:
core_unit_tmp = list(range(48,214))
secondary_motifs_tmp = [48,49,50,51,
                   59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,
                   85,86,87,88,89,90,91,92,
                   100,101,102,103,104,105,106,107,108,109,110,
                   129,130,131,132,133,134,135,136,137,138,139,140,
                   162,163,164,165,166,167,168,169,170,171,172,
                   176,177,178,179,
                   182,183,184,185,186,187,188,189,190,191,192,
                   208,209,210,211,212,213]

gaspar_0902 = [str(i) for i in list(range(48,52))+list(range(59,77))+list(range(84,92))+list(range(100,110))]

core_unit = [str(i) for i in core_unit_tmp]
secondary_motifs = [str(i) for i in secondary_motifs_tmp]

In [8]:
gaspar_0917 = list(table.columns[table.iloc[0] != ''])
print(gaspar_0917)

['48', '49', '50', '51', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '83', '84', '85', '86', '87', '88', '89', '90', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '164', '165', '166', '167', '168', '180', '181', '182', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208']


In [21]:
errors = []
RMSDS = {}
io = PDB.PDBIO()
parser = PDB.PDBParser(QUIET=True)
super_imposer = Bio.PDB.Superimposer()

target_structure = parser.get_structure('2KK6_A_SH2_N', '/home/takacsg/Documents/sh2db/Structures/Kinases/FER/2KK6/2KK6_A_SH2_N_R.pdb')
target_model = target_structure[0]
target_residues = PDB.Selection.unfold_entities(target_model, "R")

counter = 0
for i in range(len(pfam)) :
    
    cat = pfam.loc[i, "Category"]
    #gene =  pfam.loc[i, "Gene_col"]
    gene = pfam.loc[i, "Gene_domain"]
    gene_name = pfam.loc[i, "Gene_domain"]
    gene_name = gene_name.split('_')[0]
    chain =  pfam.loc[i, "PDB chain ID"]
    pdb = pfam.loc[i, "PDB ID"]
    pole =  pfam.loc[i, "Pole"]
    
    pdb_id = str(pdb+'_'+chain+'_SH2_'+pole)

    # 1. 2 pdb to compare them
    target = table.loc[('FER' , '2KK6_A_SH2_N')]
    try: 
        query = table.loc[(gene ,pdb_id)]
    except KeyError:
        continue

    # 2. Take the cells where there is an aminoacid: 
    # Target is the sequence what will be used as a template in the alignment
    target_indices=[idxi for (idxi,i) in zip(query.keys(),query.values[0]) if i in ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']]
    # Query is the structure what will be aligned to the target structure
    query_indices=[idxi for (idxi,i) in zip(target.keys(),target.values[0]) if i in ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']]


    # 3. take the intersection of the two lists
    common_indices_tmp = [i for i in target_indices if i in query_indices]
    common_indices = [i for i in common_indices_tmp if i in gaspar_0917]
    

    # 4. Searching for the residue numbers and create two lists for the structure alignment input
    target_seq = table.loc[('FER','nums')][common_indices].values[0]
    query_seq = table.loc[(gene,'nums')][common_indices].values[0]
    

    # 5. Load the PDB structures:
    if os.path.exists('/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'_R.pdb'):
        file_name =  '/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'_R.pdb'
        #counter +=1
    elif os.path.exists('/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene[:-2]+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'_R.pdb'):
        file_name = '/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene[:-2]+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'_R.pdb'
        #counter += 1
    else: 
         print("ERROR: ", pdb_id)

    query_structure = parser.get_structure(pdb_id, file_name)
    query_model = query_structure[0]

    query_residues = PDB.Selection.unfold_entities(query_model, "R")
    #print(query_residues)


    # 6. get the list of the atoms from the loaded stucture:

    target_atoms = []                    
    # Iterate all in the STAT5B model to find all residues
    for target_chain in target_model:
        for target_res in target_chain:
            #print("target RES", "\n", target_res)
            #print("target get id", target_res.get_id()) # ITT MÉG MEG VANNAK A RESIDUE-K
            # If the STAT5B residue is in the query_seq add it to the STAT5B atoms list
            if str(target_res.get_id()[1]) in target_seq:
                #print("target get id", target_res.get_id())
                target_atoms.append(target_res['CA'])

    query_atoms = []                    
    for query_chain in query_model:
        for query_res in query_chain:
            #print("QUERY RES", "\n", query_res)
            #print("query get id", query_res.get_id()) # ITT MÉG MEG VANNAK A RESIDUE-K
            if str(query_res.get_id()[1]) in query_seq:
                #print("query get id", query_res.get_id())
                query_atoms.append(query_res['CA'])

    #try:
    super_imposer.set_atoms(target_atoms, query_atoms)
    #except:
    #    errors.append(query)
    #    continue


    # Apply rotation/translation to the target_seq atoms
    super_imposer.apply(query_structure.get_atoms())
    rmsd = super_imposer.rms

    # SAVE THE RMSD VALUES INTO A DICTIONARY:
    RMSDS[pdb_id] = rmsd

    # Save the aligned structures
    io.set_structure(query_structure)
    io.save('/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene_name+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'_SUPERPOSED.pdb') # MAPPA!!
    counter += 1                

RMSDS_df = pd.DataFrame.from_dict(RMSDS, orient='index')
RMSDS_df.to_csv('../data/RMSDS_df.csv')
print(counter)

/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:45: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:46: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:45: PerformanceWarning: indexing past lexsort d

635


/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:45: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:46: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:45: PerformanceWarning: indexing past lexsort d

In [18]:
print(pdb_id)
print(query_atoms)
print(target_atoms)

1A81_A_SH2_N
[]
[<Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>, <Atom CA>]


In [16]:
counter

249

In [19]:
cat = pfam.loc[i, "Category"]
#gene =  pfam.loc[i, "Gene_col"]
gene = pfam.loc[i, "Gene_domain"]
gene_name = pfam.loc[i, "Gene_domain"]
gene_name = gene_name.split('_')[0]
chain =  pfam.loc[i, "PDB chain ID"]
pdb = pfam.loc[i, "PDB ID"]
pole =  pfam.loc[i, "Pole"]

In [10]:
print(errors)

[                   2 3 4 5 6 7 8 9 10 11  ... 254 255 256 257 258 259 260 261  \
0     1                                   ...                                   
SYK_N 1A81_A_SH2_N                        ...                                   

                   262 263  
0     1                     
SYK_N 1A81_A_SH2_N          

[1 rows x 262 columns],                    2 3 4 5 6 7 8 9 10 11  ... 254 255 256 257 258 259 260 261  \
0     1                                   ...                                   
SYK_N 1A81_C_SH2_N                        ...                                   

                   262 263  
0     1                     
SYK_N 1A81_C_SH2_N          

[1 rows x 262 columns],                    2 3 4 5 6 7 8 9 10 11  ... 254 255 256 257 258 259 260 261  \
0     1                                   ...                                   
SYK_N 1A81_E_SH2_N                        ...                                   

                   262 263  
0     1       